In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2
from fastai.structured import *
from fastai.column_data import *
from fastai.imports import *
from fastai.dataset import *
from fastai.torch_imports import *
from fastai.column_data import *
from sklearn_pandas import DataFrameMapper
from sklearn.preprocessing import LabelEncoder , Imputer , StandardScaler
import operator
import os
np.set_printoptions(threshold=50,edgeitems=20)
PATH='data/airbnb/'

In [2]:
from datetime import datetime , date

In [3]:
!ls data/airbnb


age_gender_bkts.csv  sample_submission_NDF.csv	test_users_f.csv
concat.csv	     sessions.csv		train_users_2.csv
countries.csv	     test_users.csv		train_users_f.csv


In [4]:
table_names = ['train_users_2','sessions']
tables = [pd.read_csv(f'{PATH}{fname}.csv',low_memory=False) for fname in table_names]

In [5]:
test = pd.read_csv(f'{PATH}test_users.csv')

In [6]:
for t in tables : display(t.head())

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
0,gxn3p5htnn,2010-06-28,20090319043255,NaN,-unknown-,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF
1,820tgsjxq7,2011-05-25,20090523174809,NaN,MALE,38.0,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF
2,4ft3gnwmtx,2010-09-28,20090609231247,2010-08-02,FEMALE,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US
3,bjjt8pjhuk,2011-12-05,20091031060129,2012-09-08,FEMALE,42.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other
4,87mebub9p4,2010-09-14,20091208061105,2010-02-18,-unknown-,41.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US


,user_id,action,action_type,action_detail,device_type,secs_elapsed
0,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,319.0
1,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,67753.0
2,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,301.0
3,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,22141.0
4,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,435.0


In [7]:
for t in tables : display(DataFrameSummary(t).summary())

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
count,NaN,NaN,213451,NaN,NaN,125461,NaN,213451,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,2.01309e+13,NaN,NaN,49.6683,NaN,3.26739,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,9.25372e+09,NaN,NaN,155.667,NaN,7.63771,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,2.00903e+13,NaN,NaN,1,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,2.01212e+13,NaN,NaN,28,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,2.01309e+13,NaN,NaN,34,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,2.01403e+13,NaN,NaN,43,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,NaN,NaN,2.01406e+13,NaN,NaN,2014,NaN,25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
counts,213451,213451,213451,88908,213451,125461,213451,213451,213451,213451,213451,207386,213451,213451,213451,213451
uniques,213451,1634,213451,1976,4,127,3,17,25,8,18,7,4,9,52,12


,user_id,action,action_type,action_detail,device_type,secs_elapsed
count,NaN,NaN,NaN,NaN,NaN,1.04317e+07
mean,NaN,NaN,NaN,NaN,NaN,19405.8
std,NaN,NaN,NaN,NaN,NaN,88884.2
min,NaN,NaN,NaN,NaN,NaN,0
25%,NaN,NaN,NaN,NaN,NaN,229
50%,NaN,NaN,NaN,NaN,NaN,1147
75%,NaN,NaN,NaN,NaN,NaN,8444
max,NaN,NaN,NaN,NaN,NaN,1.79998e+06
counts,10533241,10488111,9441533,9441533,10567737,10431706
uniques,135483,359,10,155,14,337661


new

In [8]:
train_users = pd.read_csv(f'{PATH}train_users_2.csv')
test_users = pd.read_csv(f'{PATH}test_users.csv')
sessions = pd.read_csv(f'{PATH}sessions.csv')
users = pd.concat([train_users, test_users], axis=0, ignore_index=True)
users.drop('date_first_booking', axis=1, inplace=True)   #test users doesnt have it
user_with_year_age_mask = users['age'] > 1000
users.loc[user_with_year_age_mask, 'age'] = 2015 - users.loc[user_with_year_age_mask, 'age']
users.loc[(users['age'] > 100) | (users['age'] < 18), 'age'] = -1   #set age limit
users['age'].fillna(-1,inplace=True)
bins = [-1, 20, 25, 30, 40, 50, 60, 75, 100]
users['age_group'] = np.digitize(users['age'], bins, right=True)

In [9]:
# number of nans
users['nans'] = np.sum([
    (users['age'] == -1),
    (users['gender'] == '-unknown-'),
    (users['language'] == '-unknown-'),
    (users['first_affiliate_tracked'] == 'untracked'),
    (users['first_browser'] == '-unknown-')
], axis=0)

In [10]:
#convert dates
users['date_account_created'] = pd.to_datetime(users['date_account_created'], errors='ignore')
users['date_first_active'] = pd.to_datetime(users['timestamp_first_active'], format='%Y%m%d%H%M%S')
date_account_created = pd.DatetimeIndex(users['date_account_created'])
date_first_active = pd.DatetimeIndex(users['date_first_active'])
#splitting into day month and year
users['day_account_created'] = date_account_created.day
users['weekday_account_created'] = date_account_created.weekday
users['week_account_created'] = date_account_created.week
users['month_account_created'] = date_account_created.month
users['year_account_created'] = date_account_created.year
users['day_first_active'] = date_first_active.day
users['weekday_first_active'] = date_first_active.weekday
users['week_first_active'] = date_first_active.week
users['month_first_active'] = date_first_active.month
users['year_first_active'] = date_first_active.year
#time lag
users['time_lag'] = (date_account_created.values - date_first_active.values).astype(int)
#drop duplicated columns
drop_list = [
    'date_account_created',
    'date_first_active',
    'timestamp_first_active'
]

users.drop(drop_list, axis=1, inplace=True)

In [11]:
#work on sessions
sessions.rename(columns = {'user_id': 'id'}, inplace=True)   #rename
#finding frequency
action_count = sessions.groupby(['id', 'action'])['secs_elapsed'].agg(len).unstack()
action_type_count = sessions.groupby(['id', 'action_type'])['secs_elapsed'].agg(len).unstack()
action_detail_count = sessions.groupby(['id', 'action_detail'])['secs_elapsed'].agg(len).unstack()
device_type_sum = sessions.groupby(['id', 'device_type'])['secs_elapsed'].agg(sum).unstack()

sessions_data = pd.concat([action_count, action_type_count, action_detail_count, device_type_sum],axis=1)
sessions_data.columns = sessions_data.columns.map(lambda x: str(x) + '_count')

# Most used device
sessions_data['most_used_device'] = sessions.groupby('id')['device_type'].max()

users = users.join(sessions_data, on='id')

#secs elapsed
secs_elapsed = sessions.groupby('id')['secs_elapsed']

secs_elapsed = secs_elapsed.agg(
    {
        'secs_elapsed_sum': np.sum,
        'secs_elapsed_mean': np.mean,
        'secs_elapsed_min': np.min,
        'secs_elapsed_max': np.max,
        'secs_elapsed_median': np.median,
        'secs_elapsed_std': np.std,
        'secs_elapsed_var': np.var,
        'day_pauses': lambda x: (x > 86400).sum(),
        'long_pauses': lambda x: (x > 300000).sum(),
        'short_pauses': lambda x: (x < 3600).sum(),
        'session_length' : np.count_nonzero
    }
)

users = users.join(secs_elapsed, on='id')

/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:32: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version


In [12]:
users.set_index('id', inplace=True)
users.loc[train_users['id']].to_csv(f'{PATH}train_users_f.csv')
users.loc[test_users['id']].drop('country_destination', axis=1).to_csv(f'{PATH}test_users_f.csv')

In [13]:
joined = pd.read_csv(f'{PATH}train_users_f.csv')
joined_test = pd.read_csv(f'{PATH}test_users_f.csv')

/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (564) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
joined.head().T.head(100)

In [ ]:
joined_test.head(7).T.head(20)

In [14]:
cat_vars = ['day_account_created','weekday_account_created','week_account_created','month_account_created','year_account_created',
           'day_first_active','weekday_first_active','week_first_active','month_first_active','year_first_active',    'gender', 'signup_method', 'signup_flow', 'language',
    'affiliate_channel', 'affiliate_provider', 'first_affiliate_tracked',
    'signup_app', 'first_device_type', 'first_browser', 'most_used_device']
contin_vars = ['age','age_group','time_lag']
n=len(joined);n

213451

In [15]:
dep = 'country_destination'
joined = joined[cat_vars+contin_vars+[dep,'id']].copy()

In [16]:
joined_test[dep]=0
joined_test = joined_test[cat_vars+contin_vars+[dep,'id']].copy()

In [17]:
for v in cat_vars:
    joined[v]=joined[v].astype('category').cat.codes

In [18]:
apply_cats(joined_test,joined)

In [19]:
for v in contin_vars:
    joined[v]=joined[v].fillna(0).astype('float32')
    joined_test[v]=joined_test[v].fillna(0).astype('float32')

In [20]:
samp_size = n
joined_samp = joined.set_index("id")

In [21]:
joined_samp.head(2)

,day_account_created,weekday_account_created,week_account_created,month_account_created,year_account_created,day_first_active,weekday_first_active,week_first_active,month_first_active,year_first_active,...,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,most_used_device,age,age_group,time_lag,country_destination
id,,,,,,,,,,,,,,,,,,,,,
gxn3p5htnn,27,0,25,5,0,18,3,11,2,0,...,4,6,2,3,8,-1,-1.0,0.0,4.024603e+16,NDF
820tgsjxq7,24,2,20,4,1,22,5,20,4,0,...,8,6,2,3,8,-1,38.0,4.0,6.318071e+16,NDF


In [22]:
df , y , nas , mapper = proc_df(joined_samp,'country_destination',do_scale= True)

AttributeError: Can only use .cat accessor with a 'category' dtype

In [ ]:
md = ColumnarDataset.from_data_frame(joined_samp,cat_vars)